On va tenter de simuler une pseudo carte scolaire. En effet, notre modélisation du lien entre taux de réussite au baccalauréat et revenu souffre probablement du fait qu'on ait associé le revenu des élèves à celui des communes de leur lycée. 
Or, des élèves viennent d'autres communes non pourvues d'établissement scolaire et le revenu médian peut être très hétérogène entre deux communes. 
On va donc essayer de refaire une carte scolaire de la manière suivante :
- Chaque commune sera associée à un point géographique. Si elle est dépourvue de lycée, on l'associera au lycée le plus proche. 
- On ponderera alors un revenu médian pour chaque lycée en fonction du revenu médian de chaque commune qui lui est associée et de la population des communes prises en compte. On posera donc l'hypothèse que la proportion de bacheliers potentiels est identique dans chaque commune (ce qui est discutable étant donné qu'il y'a une ségrégation spatiale générationnelle sur le territoire français)
- On procédera ainsi à une nouvelle régression avec les données obtenues

On doit donc créer une base de données adaptée. Il faut qu'on ait dedans
- Chaque établissement scolaire, avec son nombre de candidats et le taux de réussite par filières. 
- Chaque commune avec le revenu médian associé
- Les coordonnées géographiques des lycées et des communes 
En plus des bases de données utilisées précédemment, nous avons utilisé une base de données de La Poste pour avoir la longitude et la latitude de chaque commune. 

In [1]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 7.2 MB/s eta 0:00:00
  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-uci_yvrz
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-uci_yvrz
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 916d8bc82ea43f637047ed989fb7dbc491f56895
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.4 MB/s eta 0:00:00
  Created wheel for pynsee: filename=pynsee-0.1.2-py3-none-any.whl size=9805198 sha256=cf00516e57cb78690420fcc504938ea8e805e39123dad9b0dbeaa5ffa546fa70
  Stored in directory: /tmp/pip-ephem-wheel-cache-nr2qt8d

A - Traitement de données pour le modèle

In [114]:
#Importations des bases de données

#Base de communes

df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )
#Base des lycées
dflyc = pd.read_excel('base_def_SansDOMTOM.xlsx')
dflyc = dflyc[dflyc['Code_commune'].str.startswith('2B')==False] #On exclut la Corse pour faciliter des conversions en nombre entier futur
dflyc = dflyc[dflyc['Code_commune'].str.startswith('2A')==False]

#Base des coordonnées géographiques des communes 
dfgeo = pd.read_csv('communes-departement-region.csv')

#Base population
dfpop = pd.read_excel('ensemble.xlsx', sheet_name=4, header=7)
dfpop =dfpop[["Code département", "Code commune", "Population totale"]]
dfpop = dfpop[dfpop['Code département'].str.startswith('2B')==False] #On exclut la Corse pour faciliter des conversions en nombre entier futur
dfpop = dfpop[dfpop['Code département'].str.startswith('2A')==False]
dfpop["Code département"] = dfpop["Code département"].astype(int)
for i in range(dfpop.shape[0]):
    dfpop.loc[dfpop.index[i], "Code département"] = dfpop.loc[dfpop.index[i], "Code département"]*1000
dfpop['Code Commune INSEE'] = dfpop['Code département'] + dfpop['Code commune'] #Les codes communes seront alors compatibles avec ceux de la base de données des communes
dfpop = dfpop.drop(["Code département", "Code commune"], axis=1)

In [115]:
# NettoyageBaseGéo
dfgeo=dfgeo[dfgeo['code_postal']<97000]
dfgeo = dfgeo.drop(["code_postal", "libelle_acheminement", "ligne_5", "nom_commune_complet", "nom_departement", "nom_region","article"], axis=1)
dfgeo

#NettoyageBaseLycCom
dflyc.columns
dflyc = dflyc.drop(["Unnamed: 0", "Valeur ajoutée du taux de réussite_GNLE", "Valeur ajoutée du taux de réussite_Valeur ajoutée techno", "Nombre d'élèves présents au Bac_TOTAL_LGT","Taux de réussite bruts_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_Pro", "Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19", "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19", "coordonnee_X", "coordonnee_Y", "epsg"], axis=1)
dflyc

#Amélioration base communes
df_com.columns = df_com.columns.map('_'.join).str.strip('_')
df_com = df_com.drop(["Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19","Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"], axis=1)
df_com = df_com[df_com["Code géographique_CODGEO"].str.startswith('2A')==False]
df_com = df_com[df_com["Code géographique_CODGEO"].str.startswith('2B')==False] #On enlève la Corse pour faciliter des manipulations futures
df_com["Code géographique_CODGEO"] = df_com["Code géographique_CODGEO"].astype(int)
df_com2 = df_com.merge(dfpop, how='left', left_on = "Code géographique_CODGEO", right_on='Code Commune INSEE')
df_com2["Code géographique_CODGEO"] = df_com2["Code géographique_CODGEO"].astype(str)

In [116]:
#Fusion des bases

#Fusion population/lycée
dflyc['Code_commune'] = dflyc['Code_commune'].astype(int)
dflyc = dflyc.merge(dfpop, left_on = 'Code_commune', right_on='Code Commune INSEE')
dflyc['Code_commune'] = dflyc['Code_commune'].astype(str)

#Fusion commune/géographie
dfcomgeo  = df_com2.merge(dfgeo, how= 'left', left_on='Code géographique_CODGEO', right_on = 'code_commune_INSEE')
dfcomgeo = dfcomgeo.drop(["nom_commune_postal", "nom_commune"], axis=1)

#Fusion commune, lycée 
dfmodel = pd.merge(dflyc, dfcomgeo, how = 'outer', left_on = "Code_commune", right_on ='code_commune_INSEE')

In [123]:
dfmodelc = dfmodel.copy()

dfmodelc["Informations établissement_Ville"]= dfmodelc["Informations établissement_Ville"].fillna(0)
dfmodelcnz = dfmodelc[dfmodelc["Informations établissement_Ville"]!=0]
dfmodelcz = dfmodelc[dfmodelc["Informations établissement_Ville"]==0]
for i in range(dfmodelcnz.shape[0]):
    dfmodelcnz.loc[dfmodelcnz.index[i], "Libellé géographique_LIBGEO"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Informations établissement_Ville"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "code_commune_INSEE"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Code_commune"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "Médiane (€)_Q219_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Médiane (€)_Q219_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "latitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"latitude_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "longitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"longitude_x"]
    dfmodelcnz.loc
dfmodelc = pd.concat([dfmodelcnz, dfmodelcz])

dfmodelc = dfmodelc.drop(["Code_commune","Informations établissement_Ville","code_commune", "latitude_x", "longitude_x", "Médiane (€)_Q219_x", "Code géographique_CODGEO", "Code Commune INSEE_x", 'Population totale_x', 'Code Commune INSEE_y'], axis=1)
dfmodelc

,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Médiane (€)_Q219_y,Population totale_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,BARCELONNETTE,19070,2773.0,4019,44.388778,6.651860,4,93.0
2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0
3,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.100803,6.234041,4,93.0
4,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,DIGNE LES BAINS,19840,17240.0,4070,44.073766,6.181939,4,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villeron,24910,1265.0,95675,49.060843,2.534511,95,11.0
38872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villers-en-Arthies,27360,507.0,95676,49.085900,1.730396,95,11.0
38873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villiers-Adam,33010,872.0,95678,49.070289,2.239509,95,11.0
38874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villiers-le-Sec,25690,190.0,95682,49.074309,2.386890,95,11.0


B - élaboration du modèle

On va essayer de déterminer, pour chaque commune dépourvue de lycée, le lycée qui est le plus proche géographiquement. Ainsi, pour chaque lycée, on va grouper en son sein des communes associées. 
On posera l'hypothèse que tous les élèves vont nécessairement être scolarisés dans un lycée de leur département pour alléger l'algorithme.

In [129]:
dfmodelc['Informations établissement_Académie'] = dfmodelc['Informations établissement_Académie'].fillna(0)
dfcomsanslyc = dfmodelc[dfmodelc['Informations établissement_Académie']==0] #La base de données des communes dépourvues de lycée
dfaveclyc = dfmodelc[dfmodelc['Informations établissement_Académie']!=0]

,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Médiane (€)_Q219_y,Population totale_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
6645,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L'Abergement-Clémenciat,24210,789.0,1001,46.153426,4.926114,1,84.0
6646,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L'Abergement-de-Varey,23380,254.0,1002,46.009188,5.428017,1,84.0
6647,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ambérieux-en-Dombes,24100,1744.0,1005,45.996180,4.912273,1,84.0
6648,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ambronay,23340,2878.0,1007,46.005591,5.357607,1,84.0
6649,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ambutrix,26400,770.0,1008,45.936713,5.332809,1,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38871,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villeron,24910,1265.0,95675,49.060843,2.534511,95,11.0
38872,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villers-en-Arthies,27360,507.0,95676,49.085900,1.730396,95,11.0
38873,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villiers-Adam,33010,872.0,95678,49.070289,2.239509,95,11.0
38874,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Villiers-le-Sec,25690,190.0,95682,49.074309,2.386890,95,11.0


In [ ]:
#Définition d'une distance
def distance(IDlycee, IDcommune):
    longcom =
    latcom =
    longlyc =
    latlyc =
    d = np.sqrt((longcom, longlyc)**2 + (latcom- latlyc)**2 )
    return d

""
6645
6646
6647
6648
6649
...
38871
38872
38873
38874


In [131]:

'''
lycees = {}
for i in range(dfaveclyc.shape[0]):
    a = []
    for j in range(dfcomsanslyc.shape[0]):
        
        a.append(j)
    lycees[dfaveclyc.loc[dfaveclyc.index[i],'Identifiant_de_l_etablissement']] = a
lycees
'''

{'0040003G': [0, 1, 2],
 '0040027H': [0, 1, 2],
 '0040490L': [0, 1, 2],
 '0040007L': [0, 1, 2],
 '0040010P': [0, 1, 2],
 '0040533H': [0, 1, 2],
 '0040011R': [0, 1, 2],
 '0040023D': [0, 1, 2],
 '0133425C': [0, 1, 2],
 '0130001F': [0, 1, 2],
 '0133525L': [0, 1, 2],
 '0131319N': [0, 1, 2],
 '0130002G': [0, 1, 2],
 '0131596P': [0, 1, 2],
 '0131862D': [0, 1, 2],
 '0131391S': [0, 1, 2],
 '0131320P': [0, 1, 2],
 '0133395V': [0, 1, 2],
 '0130003H': [0, 1, 2],
 '0130006L': [0, 1, 2],
 '0130010R': [0, 1, 2],
 '0130011S': [0, 1, 2],
 '0130171R': [0, 1, 2],
 '0133274N': [0, 1, 2],
 '0131549N': [0, 1, 2],
 '0132810J': [0, 1, 2],
 '0130013U': [0, 1, 2],
 '0133244F': [0, 1, 2],
 '0133822J': [0, 1, 2],
 '0133314G': [0, 1, 2],
 '0132495S': [0, 1, 2],
 '0132276D': [0, 1, 2],
 '0131747D': [0, 1, 2],
 '0133406G': [0, 1, 2],
 '0132410Z': [0, 1, 2],
 '0130033R': [0, 1, 2],
 '0132210G': [0, 1, 2],
 '0130143K': [0, 1, 2],
 '0131484T': [0, 1, 2],
 '0133195C': [0, 1, 2],
 '0130146N': [0, 1, 2],
 '0132280H': [0,

In [140]:
dfcomsanslyc.shape[0]

32231

In [ ]:
#On remplit les listes 

for j in range(dfcomsanslyc.shape[0]):
    indcom = dfcomsanslyc.loc[j, 'code_commune_INSEE']
    a = 0
    b = 10000000000000000
    for k in range(dfaveclyc.shape[0]):
        if dfcomsanslyc.loc[j, 'code_departement'] == dfaveclyc.loc[k, 'code_departement']: 
            indlyc = dfaveclyc.loc[k, 'Identifiant_de_l_etablissement']
            a = distance(indcom,indlyc)
            if b<a:
                b = a
                IDlyc = dfaveclyc.loc[k, 'Identifiant_de_l_etablissement']
    'IDlyc'.append(indcom)

In [ ]:
#On pondère les listes pour estimer un nouveau revenu associé à chaque établissement 

for elem in lycees:
    